# DOCUMENTATION: babyalgebraist.github.io 

I'm making content for a personal website that will include details on my activities and course histories. This page will serve as a sort of 'Appendix' for my CV, with added detail should an interested person check it out. 

## General Task: Convert HTML table to DataFrame; make new HTML Table with Required Information
We want to take the raw table data (saved manually from a password protected student portal by inspecting elements with Chrome). 

We will strip away extraneous information, and make a simple list of the the institution's course name and number with a brief description or name, correcting missing or poorly formatted data with correct or better details.  

What follows are the steps taken along with some justification. 

If you have any comments, please make them on the github repository as an issue. 

Thanks a bunch! 

In [1]:
import pandas as pd

The Column names are Truncated, and or Abbreviated. Let's change that. 

In [2]:
col_names = ['Course', 'Num','Section',	'Description',	
             'Attempted','Grade','Notation',	
             'GPA','Class Avg',	'Class Size',	
             'Credits Earned','Other']

Read 'FILE_NAME' into DataFrames with pandas 'pd.read_html()'.  

Will return a list containing all the tables as DataFrames: 'df_list'

For each DataFrame, we're going to make a copy, each as a csv named for their index in the list. We expect 8. 

In [3]:
FILE_NAME = 'assets/data/course_history_raw.html'

df_list = pd.read_html(FILE_NAME)

So we made a list of the files to use to build a clean list of courses. We are only really interested in Table 2 onward. So lets remove 'table_0.csv' and 'table_1.csv', making the list only 6 long. 

In [4]:
df_list = df_list[2:]
len(df_list)

6

In [5]:
files = []
for i, df in enumerate(df_list):
#     print(df)
    df.to_csv('assets/data/table_{}.csv'.format(i),
              index=False)
    files.append('assets/data/table_{}.csv'.format(i))

files

['assets/data/table_0.csv',
 'assets/data/table_1.csv',
 'assets/data/table_2.csv',
 'assets/data/table_3.csv',
 'assets/data/table_4.csv',
 'assets/data/table_5.csv']

Inpsect the DataFrames...

In [6]:
i = 0 
for table in df_list:
    print("\nTable number {} \n\n".format(i), table)
    i+=1


Table number 0 

   Course  Course.1 Course.2                  Description  Attempted Grade  \
0   FRAN       305        B          COMMUNICATION ORALE        3.0    B+   
1   GEOL       210        X    INTRODUCTION TO THE EARTH        3.0    A-   
2   MAST       217        A  INTRO/MATHEMATICAL THINKING        3.0    A+   
3   MAST       219        A    MULTIVARIABLE CALCULUS II        3.0     C   
4   MECH       211        Y     MECHANICAL ENGR. DRAWING        3.5    B-   

   Notation  GPA  Class Avg  Class Size  Program Credits Earned  Other  
0       NaN  3.3       3.12          26                     3.0    NaN  
1       NaN  3.7       3.25          69                     3.0    NaN  
2       NaN  4.3       2.94          49                     3.0    NaN  
3       NaN  2.0       1.69          17                     3.0    NaN  
4       NaN  2.7       2.62          70                     3.5    NaN  

Table number 1 

   Course  Course.1 Course.2                  Description  Att

First, we'll sneakily save the 'names' for the DataFrames in a DataFrames as strings: `df_<cnt>`

Then concatenate the DataFrames

In a loop, we'll read in each of the tables saved as CSVs in the last step. 
Then drop unnecessary column: 'Section' in place

In [7]:
cnt = len(files)  # counter for loop
dict_of_df = {} # initialize empty dictionary

for i in range(cnt):
    dict_of_df['df_{}'.format(i)] = pd.read_csv('assets/data/table_{}.csv'.format(i), 
                                                header=None,
                                                skiprows=1,usecols=[0,1,2,3], 
                                                names=col_names[:4])
dfs_list = list(dict_of_df.keys())

df = pd.concat(dict_of_df, ignore_index=True)
# display(df)
df.drop(columns=['Section'], inplace=True)
display(df)

,Course,Num,Description
0,FRAN,305,COMMUNICATION ORALE
1,GEOL,210,INTRODUCTION TO THE EARTH
2,MAST,217,INTRO/MATHEMATICAL THINKING
3,MAST,219,MULTIVARIABLE CALCULUS II
4,MECH,211,MECHANICAL ENGR. DRAWING
5,GEOL,208,"THE EARTH, MOON AND PLANETS"
6,MATH,252,LINEAR ALGEBRA II
7,MATH,365,ANALYSIS II
8,STAT,249,PROBABILITY I
9,MATH,366,COMPLEX ANALYSIS I


See that there are some extraneous 'Course Topic' etries set by the student portal to handle topics classes or courses taken by Inter University Transfer (Crépuq here in Quebec). These rows should be removed.  We'll use a mask to find them all. 

See the 15th record is a topic course. We need to rescue the data in the description column, put it in 'Course Number' column as 'MATH 457'  and set the description to 'Fields and Galois Theory'

Similarly, 
-  update 'Description' to `df.loc[index_from_mask - 1, 'Description] = df.loc[index_from_mask, 'Description']`
-  but with some special attention paid to the 'Course' and 'Num' information. (see below)
- likewise for the other records. No special attention is required

In [8]:
# The boolean mask to find index of location of the 'Course Topics:' value

mask = df.Course == 'Course Topic:'
topics_ind = list(df[mask].index)
topics = list(df[mask].Num.values)

topics_ilocs = list(zip(topics_ind, topics))

topics_ilocs

[(13, 'Course Topic:'),
 (18, 'Course Topic:'),
 (23, 'Course Topic:'),
 (25, 'Course Topic:'),
 (27, 'Course Topic:')]

Reminder: the record for 'Honors Algebra 4' has to have some extra lifting.  Here it is for reference. 

In [9]:
df.loc[24:25, 'Course':'Description']

,Course,Num,Description
24,INTU,MJKY,HONOURS ALGEBRA 4
25,Course Topic:,Course Topic:,MATH 457 MCGILL


This should be self-explanitory. 

In [10]:
for ind, value in iter(topics_ilocs):
    if df.loc[ind - 1, 'Course'] == 'INTU':
        df.loc[ind - 1, 'Course'] = 'MATH'
        df.loc[ind - 1, 'Num'] = '457'
        df.loc[ind - 1, 'Description'] = 'Fields and Galois Theory'.upper()
        df.drop([ind], inplace=True)
        print('\n')
        print(df.loc[ind - 1])
    else:
        df.loc[ind - 1, 'Description'] = 'TOPIC: ' + df.loc[ind, 'Description']
        df.drop([ind], inplace=True)
        print('\n', df.loc[ind - 1], '\n')


 Course                                      MATH
Num                                          475
Description    TOPIC: DISCRETE DYNAMICAL SYSTEMS
Name: 12, dtype: object 


 Course                      MATH
Num                          494
Description    TOPIC: SET THEORY
Name: 17, dtype: object 


 Course                                   MATH
Num                                       494
Description    TOPIC: ALGEBRAIC NUMBER THEORY
Name: 22, dtype: object 



Course                             MATH
Num                                 457
Description    FIELDS AND GALOIS THEORY
Name: 24, dtype: object

 Course                           MATH
Num                               494
Description    TOPIC: RANDOM MATRICES
Name: 26, dtype: object 



In [11]:
df.reset_index(inplace=True, drop=True)
df

,Course,Num,Description
0,FRAN,305,COMMUNICATION ORALE
1,GEOL,210,INTRODUCTION TO THE EARTH
2,MAST,217,INTRO/MATHEMATICAL THINKING
3,MAST,219,MULTIVARIABLE CALCULUS II
4,MECH,211,MECHANICAL ENGR. DRAWING
5,GEOL,208,"THE EARTH, MOON AND PLANETS"
6,MATH,252,LINEAR ALGEBRA II
7,MATH,365,ANALYSIS II
8,STAT,249,PROBABILITY I
9,MATH,366,COMPLEX ANALYSIS I


Now, that we have the 'Descriptions' more or less okay, we'll cast the 'Num' column as a string to concatenate the strings they contain.  

We do this because we want a single column with the Departement code and the course code as one strong for reading. 

In [12]:
df['Num'] = df.Num.astype(str);
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 3 columns):
Course         23 non-null object
Num            23 non-null object
Description    23 non-null object
dtypes: object(3)
memory usage: 632.0+ bytes


Now, we actually do the concatenation, making the new column 'Course Number' then drop the old columns, 'Course' and 'Num' in place, before reordering the columns for readability, and dropping the extra 'Topics' rows 

In [13]:
df['Course Number'] = (df['Course'] + ' ' + df['Num'])
df.drop(columns=['Course', 'Num'], inplace=True)
df = df[['Course Number', 'Description']]

df

,Course Number,Description
0,FRAN 305,COMMUNICATION ORALE
1,GEOL 210,INTRODUCTION TO THE EARTH
2,MAST 217,INTRO/MATHEMATICAL THINKING
3,MAST 219,MULTIVARIABLE CALCULUS II
4,MECH 211,MECHANICAL ENGR. DRAWING
5,GEOL 208,"THE EARTH, MOON AND PLANETS"
6,MATH 252,LINEAR ALGEBRA II
7,MATH 365,ANALYSIS II
8,STAT 249,PROBABILITY I
9,MATH 366,COMPLEX ANALYSIS I


Looks great!

Save them to files: CSV for later usage; html for incorporation into the website. 

In [16]:
df.to_csv('assets/data/course_history.csv')

In [17]:
df.to_html('assets/data/course_history_temp.html', index=False)

Thoughts: 
As I have two degrees at 4 institutions, consider adding these and columns that state year taken, at which institution. 